# **Track and Count Vehicles with YOLOv8**

This Jupyter Notebook is dedicated to tracking and counting vehicles in video footage using advanced object detection models. The primary aim is to accurately identify, follow, and enumerate different types of vehicles across video frames. This analysis is pivotal for traffic flow management, urban planning, and automated surveillance systems. Leveraging deep learning models, specifically designed for object detection in videos, this notebook outlines a comprehensive approach for real-time vehicle tracking and counting.

## **GPU Status Check**

We begin by checking the availability and status of our GPU, which is crucial for the computationally intensive tasks of video processing and running the YOLOv8 model. The nvidia-smi command gives us a snapshot of the GPU's model, memory usage, and active processes, ensuring our setup is ready for the subsequent operations.

In [ ]:
!nvidia-smi

## **Importing Libraries and Setting Up the Workspace**

After downloading the necessary video file for vehicle tracking and counting, this cell focuses on importing essential Python libraries and modules that will be used throughout the notebook. Additionally, it reaffirms the home directory setup, ensuring all file paths are correctly managed.

In [ ]:
import os
import sys
from typing import List

import numpy as np
from IPython.display import display, clear_output
from PIL import Image
from tqdm.notebook import tqdm

HOME = os.getcwd()
print(HOME)

## **Mounting the Drive**



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls "/content/drive/My Drive/"

## **Setting the Source Video Path**

Define the path to the video file used for vehicle tracking and counting analysis, ensuring it's correctly located within the notebook's home directory for easy access during processing.

In [ ]:
%cd {HOME}
SOURCE_VIDEO_PATH = f"{HOME}/drive/MyDrive/demo_trim.mp4"


## **Installing Ultralytics and Preparing the Environment**

Install the ultralytics package, clear the output to maintain a clean notebook, and verify the installation by checking the environment setup.

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()


## **Installing a Specific Version of the Supervision Library**

To ensure compatibility and access to specific features required for vehicle tracking and counting, this step involves installing version 0.1.0 of the supervision library. Following the installation, the output is cleared to maintain a clean notebook workspace, and the installed version of supervision is verified.



In [ ]:
!pip install supervision==0.13.0


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.13.0



## **Specifying the YOLOv8 Model for Vehicle Detection**

This line of code sets the model to be used for vehicle detection to YOLOv8x, indicating the specific version of the YOLO model optimized for accuracy and performance in detecting objects within video frames.

In [ ]:
MODEL = "yolov8x.pt"


## **Loading and Optimizing the YOLOv8x Model for Vehicle Detection**

This section involves importing the YOLO class from the ultralytics package, initializing the YOLO model with the specified model configuration, and applying model optimization techniques to enhance performance.

In [ ]:
from ultralytics import YOLO
model = YOLO(MODEL)
model.fuse()

## **Configuring Class IDs for Vehicle Detection**

This step involves setting up a dictionary to map class IDs to their corresponding class names for the YOLOv8x model and specifying the class IDs of interest for vehicle detection, such as cars, motorcycles, buses, and trucks.

In [ ]:
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names

# class_ids of interest - car, motorcycle, bus and truck
selected_classes = [2, 3, 5, 7]

## **Initializing Video Processing and Detection**

This section outlines the steps to generate frames from the video, annotate detected vehicles, and display the processed frame. It involves setting up a frame generator, initializing a BoxAnnotator for drawing detections, performing a model prediction on the first frame, and visually presenting the results.

In [ ]:
import supervision as sv
import numpy as np

In [ ]:
# create frame generator
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
# create instance of BoxAnnotator
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
# acquire first video frame
iterator = iter(generator)
frame = next(iterator)
# model prediction on single frame and conversion to supervision Detections
results = model(frame)[0]

# convert to Detections
detections = sv.Detections.from_ultralytics(results)
# only consider class id from selected_classes define above
detections = detections[np.isin(detections.class_id, selected_classes)]

# format custom labels
labels = [
    f"{CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
    for _, _, confidence, class_id, _ in detections
]

# annotate and display frame
anotated_frame=box_annotator.annotate(scene=frame, detections=detections, labels=labels)

%matplotlib inline
sv.plot_image(anotated_frame, (16,16))

## **Configuring Line Detection and Output Video Path**

This step specifies settings for a line used in vehicle counting and defines the path for saving the processed video with annotations.



In [ ]:
# settings
LINE_START = sv.Point(50, 1500)
LINE_END = sv.Point(3840-50, 1500)

TARGET_VIDEO_PATH = f"{HOME}/drive/MyDrive/demo_vehicle_result.mp4"

## **Extracting Video Information**

This step involves obtaining essential information about the source video, such as frame rate, resolution, and total frames. This information serves as the basis for subsequent video processing steps.

In [ ]:
sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

VideoInfo(width=1280, height=720, fps=59, total_frames=1208)

## **Implementing Vehicle Tracking and Counting**

This section combines vehicle detection, tracking, and counting into a single workflow, utilizing BYTETracker for tracking, annotations for visual feedback, and a line counter for vehicle counting.

**Initialize Tracking and Annotation Tools:** bold text Set up BYTETracker, video metadata retrieval, frame generation, line counting, and annotations for boxes and lines.

**Process Video Frames:**
- Detect vehicles using YOLO and filter detections by class.
- Update BYTETracker with detections for tracking.
- Match detections to tracks and update tracker IDs.
- Count vehicles crossing a predefined line.
Annotate frames with detection boxes and line crossings.

**Output:** Write annotated frames to a target video file.

In [ ]:
from tqdm.notebook import tqdm

# create BYTETracker instance
byte_tracker = sv.ByteTrack(track_thresh= 0.25, track_buffer = 30,match_thresh = 0.8,frame_rate =30)

# create VideoInfo instance
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# create frame generator
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

# create LineZone instance, it is previously called LineCounter class
line_zone = sv.LineZone(start=LINE_START, end=LINE_END)

# create instance of BoxAnnotator
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)

# create LineZoneAnnotator instance, it is previously called LineCounterAnnotator class
line_zone_annotator = sv.LineZoneAnnotator(thickness=4, text_thickness=4, text_scale=2)

progress_bar = tqdm(total=video_info.total_frames, desc="Processing Video")

# define call back function to be used in video processing
def callback(frame: np.ndarray, index:int) -> np.ndarray:
  global process_bar
  # model prediction on single frame and conversion to supervision Detections
  results = model(frame)[0]
  detections = sv.Detections.from_ultralytics(results)
  # only consider class id from selected_classes define above
  detections = detections[np.isin(detections.class_id, selected_classes)]
  # tracking detections
  detections = byte_tracker.update_with_detections(detections)
  labels = [
     f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
     for _, _, confidence, class_id, tracker_id
     in detections
  ]
  # update progress bar
  progress_bar.update(1)

  box_annotated_frame=box_annotator.annotate(scene=frame.copy(),
                                detections=detections,
                                labels=labels)
  # update line counter
  line_zone.trigger(detections)
  # return frame with box and line annotated result
  return  line_zone_annotator.annotate(box_annotated_frame, line_counter=line_zone)

# process the whole video
sv.process_video(
    source_path = SOURCE_VIDEO_PATH,
    target_path = TARGET_VIDEO_PATH,
    callback=callback
)

progress_bar.close()